# Pommerman V4 Training in smaller central region

This notebook demonstrates how to train Pommerman agents. Please let us know at support@pommerman.com if you run into any issues.

In [1]:
import os
import sys
import numpy as np
import time

import pommerman
from pommerman.agents import SimpleAgent, RandomAgent, PlayerAgent, BaseAgent, StaticAgent, RandomAgentNoBomb, RandomAgent, SmartRandomAgentNoBomb
from pommerman.configs import lesson1_1v1_env, lesson2a_1v1_env, lesson2e_1v1_env, lesson3a_1v1_env, lesson3e_1v1_env, lesson4a_1v1_env
from pommerman.envs.v0 import Pomme as Pomme_v0
from pommerman.envs.v4 import Pomme as Pomme_v4
from pommerman.characters import Bomber
from pommerman import utility
from pommerman import agents
from pommerman import envs
from pommerman import constants
from pommerman import characters

# print all env configs
print(pommerman.REGISTRY)

['AdvancedLesson_1v1-v0', 'AdvancedLessonTeam-v0', 'PommeFFACompetition-v0', 'PommeFFACompetitionFast-v0', 'PommeFFAFast-v0', 'PommeFFA-v1', 'PommeFFAFast-v3', 'PommeFFAFast-v4', 'Lesson1_1v1-v0', 'Lesson1-v0', 'Lesson2Team-v0', 'Lesson2a_1v1-v0', 'Lesson2b_1v1-v0', 'Lesson2bTeam-v0', 'Lesson2c_1v1-v0', 'Lesson2c-v0', 'Lesson2d_1v1-v0', 'Lesson2d-v0', 'Lesson2e-v0', 'Lesson2eTeam-v0', 'Lesson3Team-v0', 'Lesson3a_1v1-v0', 'Lesson3b_1v1-v0', 'Lesson3bTeam-v0', 'Lesson3c-v0', 'Lesson3cTeam-v0', 'Lesson3d_1v1-v0', 'Lesson3dTeam-v0', 'OneVsOne-v0', 'PommeRadioCompetition-v2', 'PommeRadio-v2', 'Simple-v0', 'SimpleRandomTeam-v0', 'SimpleTeam-v0', 'PommeTeamCompetition-v0', 'PommeTeamCompetitionFast-v0', 'PommeTeamCompetition-v1', 'PommeTeam-v0', 'PommeTeamFast-v0', 'PommeTeamSimple-v0']


# Train with stable baseline

In [2]:
import gym

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines import PPO2

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Inherit pommerman env and make it compatible with stable-baseline

In [3]:
# class CustomPomme(Pomme_v4):
class CustomPomme(Pomme_v0):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.obs_raw = None # store the raw version of observation
        self.training_idx = 1 # idx of the agent being trained
    
    # function to flatten pommerman observation
    def _transform_obs(self, obs_raw):
        obs_training = obs_raw[self.training_idx] # default the first agent to be trained

        # construct flattened observation
        obs = [
            *np.array(obs_training["board"]).reshape(-1),
            *np.array(obs_training["bomb_blast_strength"]).reshape(-1),
            *np.array(obs_training["bomb_life"]).reshape(-1),
            *np.array(obs_training["position"]).reshape(-1),
            obs_training["ammo"],
            obs_training["blast_strength"],
            obs_training["can_kick"],
            obs_training["teammate"].value,
            obs_training["enemies"][0].value,
            
            # uncommon if training 1 v 1
            obs_training["enemies"][0].value,
            obs_training["enemies"][0].value,
            
            # uncommon if training 2 v 2
#             obs_training["enemies"][1].value,
#             obs_training["enemies"][2].value,
        ]
        return obs
    
    def get_obs_raw(self):
        return self.obs_raw

    def step(self, action_training):
        action_nontraining = self.act(self.obs_raw)
        actions = [*action_nontraining, action_training]
        obs_raw, reward, done, info = super().step(actions)
        self.obs_raw = obs_raw
        return self._transform_obs(obs_raw), reward[self.training_idx], done, info
    
    def reset(self):
        obs_raw = super().reset()
        self.obs_raw = obs_raw
        return self._transform_obs(obs_raw)
    
    def render(self,
               mode=None,
               close=False,
               record_pngs_dir=None,
               record_json_dir=None,
               do_sleep=True):
        super().render(mode=mode,
                       close=close,
                       record_pngs_dir=record_pngs_dir,
                       record_json_dir=record_json_dir,
                       do_sleep=do_sleep)

# Custom CNN Policy

In [10]:
import tensorflow as tf

from stable_baselines.a2c.utils import linear
from stable_baselines.common.policies import ActorCriticPolicy

class CustomCNN(ActorCriticPolicy):
    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=False, **kwargs):
        super(CustomCNN, self).__init__(sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=reuse)
        size=11
        bp = 3*size**2 #board partition
        with tf.variable_scope("model", reuse=reuse):
            obs = self.processed_obs
            self.board1, self.misc = tf.split(obs, [bp, -1], 1)
            
            self.board = tf.reshape(self.board1, (-1, size, size, 3))
            self.conv1 = tf.layers.conv2d(inputs=self.board, filters=64, kernel_size=2, padding='VALID', activation=tf.nn.relu, name='conv1')
            self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=32, kernel_size=2, padding='VALID', activation=tf.nn.relu, name='conv2')
            self.fc0 = tf.contrib.layers.flatten(self.conv2)
            self.fc1 = tf.concat((self.fc0, self.misc), -1)
            self.fc1 = tf.layers.dense(self.fc1, 1024, name = 'fc1')
            self.actions = tf.layers.dense(self.fc1, 6)   
            self.valueUM = tf.layers.dense(self.fc1, 128) #??

            self._proba_distribution, self._policy, self.q_value = \
                self.pdtype.proba_distribution_from_latent(self.actions, self.valueUM, init_scale=0.01)

        self._value_fn = linear(self.valueUM, 'vf', 1)
        self._setup_init()

    def step(self, obs, state=None, mask=None, deterministic=False):
        if deterministic:
            action, value, neglogp = self.sess.run([self.deterministic_action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        else:
            action, value, neglogp = self.sess.run([self.action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        return action, value, self.initial_state, neglogp

    def proba_step(self, obs, state=None, mask=None):
        return self.sess.run(self.policy_proba, {self.obs_ph: obs})

    def value(self, obs, state=None, mask=None):
        return self.sess.run(self.value_flat, {self.obs_ph: obs})

In [5]:
# def team_v3_fast_env():
#     """Start up a FFA config with the default settings."""
#     env = CustomPomme
#     game_type = constants.GameType.Team
#     env_entry_point = 'CustomPomme'
#     env_id = 'PommeTeamFast-v3'
#     env_kwargs = {
#         'game_type': game_type,
#         'board_size': 8,
#         'num_rigid': 0,
#         'num_wood': 0,
#         'num_items': 0,
#         'max_steps': constants.MAX_STEPS,
#         'render_fps': 1000,
#         'env': env_entry_point,
#     }
#     agent = characters.Bomber
#     return locals()

def one_vs_one_v3_env():
    """Start up a FFA config with the default settings."""
    env = CustomPomme
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'CustomPomme'
    env_id = 'PommeOneVsOneFast-v3'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'num_rigid': 0,
        'num_wood': 4,
        'num_items': 0,
        'max_steps': constants.MAX_STEPS,
        'render_fps': 1000,
        'rand_agent_pos': True,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

def one_vs_one_v4_env():
    """Start up a FFA config with the default settings."""
    env = CustomPomme
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'CustomPomme'
    env_id = 'PommeOneVsOneFast-v4'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'free_board_size': 8,
        'num_rigid': 0,
        'num_wood': 32,
        'num_items': 0,
        'max_steps': constants.MAX_STEPS,
        'render_fps': 1000,
        'rand_agent_pos': True,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

In [16]:
# Instantiate the environment

config = lesson1_1v1_env()
env_pom = CustomPomme(**config["env_kwargs"])


# config agents
agents = []

# Add simple agents
for agent_id in range(1):
    agents.append(StaticAgent(config["agent"](agent_id, config["game_type"])))

# add player agent(to train)
agents.append(PlayerAgent(config["agent"](1, config["game_type"])))

env_pom.set_agents(agents)
env_pom.set_training_agent(agents[1].agent_id)
env_pom.set_init_game_state(None)

# Seed and reset the environment
env_pom.seed(0)

[0]

In [17]:
# log function during training, implement if needed
def log(local_var, global_var):
    pass
#     display(local_var)
#     display(global_var)

In [24]:
import time
n_cpu = 1
env = DummyVecEnv([lambda: env_pom for i in range(n_cpu)])

# model = PPO2(policy=CustomCNN, 
#              env=env, 
#              verbose=1, 
#              n_steps=3000, # batch_size = n_step * num_env
#              ent_coef=0.001, # entropy coefficient
#              tensorboard_log="./PPO2_CNNPolicy_tensorboard/")
             
# model = PPO2(policy='MlpPolicy', 
#              env=env, 
#              verbose=1, 
#              n_steps=3000, # batch_size = n_step * num_env
#              ent_coef=0.001, # entropy coefficient
#              tensorboard_log="./PPO2_pommerman_tensorboard/")

model = PPO2.load(load_path='PPO2_MlpPolicy_lesson1_Stable_4M', 
                  env=env)
model.tensorboard_log = "./PPO2_pommerman_tensorboard/"

startTime = time.time()
model = model.learn(total_timesteps=4000000, # num_update = total_timesteps // batch_size
                    callback=log)
endTime = time.time()
elapsedTime = endTime - startTime
print(elapsedTime)

model.save("PPO2_MlpPolicy_lesson1_Stable_8M")

--------------------------------------
| approxkl           | 0.003051533   |
| clipfrac           | 0.02933333    |
| explained_variance | 0.201         |
| fps                | 1492          |
| n_updates          | 1             |
| policy_entropy     | 1.1539589     |
| policy_loss        | -0.0068770084 |
| serial_timesteps   | 3000          |
| time_elapsed       | 3.81e-06      |
| total_timesteps    | 3000          |
| value_loss         | 0.010225196   |
--------------------------------------
-------------------------------------
| approxkl           | 0.0026689176 |
| clipfrac           | 0.028083334  |
| explained_variance | 0.224        |
| fps                | 1666         |
| n_updates          | 2            |
| policy_entropy     | 1.1885583    |
| policy_loss        | -0.003852942 |
| serial_timesteps   | 6000         |
| time_elapsed       | 2.01         |
| total_timesteps    | 6000         |
| value_loss         | 0.014788186  |
-------------------------------------

--------------------------------------
| approxkl           | 0.0008716986  |
| clipfrac           | 0.007916667   |
| explained_variance | 0.339         |
| fps                | 1707          |
| n_updates          | 18            |
| policy_entropy     | 1.0582573     |
| policy_loss        | -0.0032159938 |
| serial_timesteps   | 54000         |
| time_elapsed       | 31.2          |
| total_timesteps    | 54000         |
| value_loss         | 0.009130327   |
--------------------------------------
-------------------------------------
| approxkl           | 0.0038164894 |
| clipfrac           | 0.054500002  |
| explained_variance | 0.181        |
| fps                | 1669         |
| n_updates          | 19           |
| policy_entropy     | 1.0709889    |
| policy_loss        | -0.006219391 |
| serial_timesteps   | 57000        |
| time_elapsed       | 33           |
| total_timesteps    | 57000        |
| value_loss         | 0.010033565  |
-------------------------------------

--------------------------------------
| approxkl           | 0.0015151575  |
| clipfrac           | 0.013583333   |
| explained_variance | 0.554         |
| fps                | 1636          |
| n_updates          | 35            |
| policy_entropy     | 1.072666      |
| policy_loss        | -0.0058092363 |
| serial_timesteps   | 105000        |
| time_elapsed       | 62.1          |
| total_timesteps    | 105000        |
| value_loss         | 0.013309544   |
--------------------------------------
--------------------------------------
| approxkl           | 0.0031968858  |
| clipfrac           | 0.045         |
| explained_variance | 0.306         |
| fps                | 1628          |
| n_updates          | 36            |
| policy_entropy     | 1.0949414     |
| policy_loss        | -0.0044374205 |
| serial_timesteps   | 108000        |
| time_elapsed       | 63.9          |
| total_timesteps    | 108000        |
| value_loss         | 0.009851315   |
-------------------------

--------------------------------------
| approxkl           | 0.0010929266  |
| clipfrac           | 0.009083333   |
| explained_variance | 0.807         |
| fps                | 1614          |
| n_updates          | 52            |
| policy_entropy     | 1.1602792     |
| policy_loss        | -0.0039409935 |
| serial_timesteps   | 156000        |
| time_elapsed       | 93.3          |
| total_timesteps    | 156000        |
| value_loss         | 0.011659163   |
--------------------------------------
-------------------------------------
| approxkl           | 0.0015833462 |
| clipfrac           | 0.012416666  |
| explained_variance | 0.681        |
| fps                | 1602         |
| n_updates          | 53           |
| policy_entropy     | 1.1355269    |
| policy_loss        | -0.002823684 |
| serial_timesteps   | 159000       |
| time_elapsed       | 95.1         |
| total_timesteps    | 159000       |
| value_loss         | 0.00866971   |
-------------------------------------

--------------------------------------
| approxkl           | 0.0014597855  |
| clipfrac           | 0.011         |
| explained_variance | 0.867         |
| fps                | 1650          |
| n_updates          | 69            |
| policy_entropy     | 1.1079142     |
| policy_loss        | -0.0015565425 |
| serial_timesteps   | 207000        |
| time_elapsed       | 124           |
| total_timesteps    | 207000        |
| value_loss         | 0.0087893065  |
--------------------------------------
-------------------------------------
| approxkl           | 0.0013234331 |
| clipfrac           | 0.009500001  |
| explained_variance | 0.453        |
| fps                | 1735         |
| n_updates          | 70           |
| policy_entropy     | 1.0850458    |
| policy_loss        | -0.002597121 |
| serial_timesteps   | 210000       |
| time_elapsed       | 126          |
| total_timesteps    | 210000       |
| value_loss         | 0.00916257   |
-------------------------------------

...

--------------------------------------
| approxkl           | 0.0015867858  |
| clipfrac           | 0.016916666   |
| explained_variance | 0.912         |
| fps                | 1400          |
| n_updates          | 1259          |
| policy_entropy     | 0.7224007     |
| policy_loss        | -0.0027339878 |
| serial_timesteps   | 3777000       |
| time_elapsed       | 8.21e+03      |
| total_timesteps    | 3777000       |
| value_loss         | 0.0075591025  |
--------------------------------------
--------------------------------------
| approxkl           | 0.002745967   |
| clipfrac           | 0.043166667   |
| explained_variance | 0.848         |
| fps                | 1395          |
| n_updates          | 1260          |
| policy_entropy     | 0.80724245    |
| policy_loss        | -0.0039709173 |
| serial_timesteps   | 3780000       |
| time_elapsed       | 8.21e+03      |
| total_timesteps    | 3780000       |
| value_loss         | 0.0080998605  |
-------------------------

--------------------------------------
| approxkl           | 0.0007425808  |
| clipfrac           | 0.0070833336  |
| explained_variance | 0.862         |
| fps                | 1285          |
| n_updates          | 1276          |
| policy_entropy     | 0.68077874    |
| policy_loss        | -0.0012359559 |
| serial_timesteps   | 3828000       |
| time_elapsed       | 8.25e+03      |
| total_timesteps    | 3828000       |
| value_loss         | 0.0081814425  |
--------------------------------------
--------------------------------------
| approxkl           | 0.00039470254 |
| clipfrac           | 0.0030833334  |
| explained_variance | 0.926         |
| fps                | 1303          |
| n_updates          | 1277          |
| policy_entropy     | 0.669392      |
| policy_loss        | -0.0017051109 |
| serial_timesteps   | 3831000       |
| time_elapsed       | 8.25e+03      |
| total_timesteps    | 3831000       |
| value_loss         | 0.00749816    |
-------------------------

-------------------------------------
| approxkl           | 0.002228837  |
| clipfrac           | 0.03125      |
| explained_variance | 0.887        |
| fps                | 1405         |
| n_updates          | 1293         |
| policy_entropy     | 0.8233677    |
| policy_loss        | -0.003524066 |
| serial_timesteps   | 3879000      |
| time_elapsed       | 8.29e+03     |
| total_timesteps    | 3879000      |
| value_loss         | 0.008832895  |
-------------------------------------
--------------------------------------
| approxkl           | 0.0005979246  |
| clipfrac           | 0.0019166665  |
| explained_variance | 0.87          |
| fps                | 1344          |
| n_updates          | 1294          |
| policy_entropy     | 0.8347462     |
| policy_loss        | -0.0014280087 |
| serial_timesteps   | 3882000       |
| time_elapsed       | 8.29e+03      |
| total_timesteps    | 3882000       |
| value_loss         | 0.008522298   |
--------------------------------------

--------------------------------------
| approxkl           | 0.0013914912  |
| clipfrac           | 0.00925       |
| explained_variance | 0.894         |
| fps                | 1346          |
| n_updates          | 1310          |
| policy_entropy     | 0.92076737    |
| policy_loss        | -0.0014953044 |
| serial_timesteps   | 3930000       |
| time_elapsed       | 8.32e+03      |
| total_timesteps    | 3930000       |
| value_loss         | 0.008023205   |
--------------------------------------
--------------------------------------
| approxkl           | 0.0022563906  |
| clipfrac           | 0.028833332   |
| explained_variance | 0.92          |
| fps                | 1299          |
| n_updates          | 1311          |
| policy_entropy     | 0.8803272     |
| policy_loss        | -0.0026474928 |
| serial_timesteps   | 3933000       |
| time_elapsed       | 8.33e+03      |
| total_timesteps    | 3933000       |
| value_loss         | 0.008032722   |
-------------------------

-------------------------------------
| approxkl           | 0.0018791371 |
| clipfrac           | 0.021583334  |
| explained_variance | 0.919        |
| fps                | 1388         |
| n_updates          | 1327         |
| policy_entropy     | 0.90923536   |
| policy_loss        | -0.003639622 |
| serial_timesteps   | 3981000      |
| time_elapsed       | 8.36e+03     |
| total_timesteps    | 3981000      |
| value_loss         | 0.007914837  |
-------------------------------------
--------------------------------------
| approxkl           | 0.0020186717  |
| clipfrac           | 0.021083333   |
| explained_variance | 0.885         |
| fps                | 1396          |
| n_updates          | 1328          |
| policy_entropy     | 0.94026804    |
| policy_loss        | -0.0041077198 |
| serial_timesteps   | 3984000       |
| time_elapsed       | 8.36e+03      |
| total_timesteps    | 3984000       |
| value_loss         | 0.0077917427  |
--------------------------------------

In [25]:
# del model # remove to demonstrate saving and loading
model = PPO2.load(load_path="PPO2_MlpPolicy_lesson1_Stable_4M")

n_cpu = 1
env = DummyVecEnv([lambda: env_pom for i in range(n_cpu)])
model.envs = env

# test the learned model
num_win = 0
num_tie = 0
num_lose = 0
total = 10 # number of playouts
for i_episode in range(total):
    obs = env.reset()
    done = False
    info = None
    while not done:
        env.render()
        action_training, _states = model.predict(obs)
#         print(action_training)
        obs, rewards, dones, infos = env.step(action_training)
#         print(infos)
        done = dones[0]
        info = infos[0]
        time.sleep(0.1)
    print('Episode {} finished'.format(i_episode))
    if(info["result"].value == 0):
        if(1 in info["winners"]):
            num_win+=1
        else:
            num_lose+=1
    elif(info["result"].value == 2):
        num_tie+=1
#     print(info)
env.close()
print("Win ", num_win, "/", total, " games")
print("Tie ", num_tie, "/", total, " games")
print("Lose ", num_lose, "/", total, " games")


Loading a model without an environment, this model cannot be trained until it has a valid environment.
Episode 0 finished
Episode 1 finished
Episode 2 finished
Episode 3 finished
Episode 4 finished
Episode 5 finished
Episode 6 finished
Episode 7 finished
Episode 8 finished
Episode 9 finished
Win  0 / 10  games
Tie  5 / 10  games
Lose  5 / 10  games


# Export JSON or PNG file

In [ ]:
#create directories for saving renders; each episode should be in its own directory if recording multiple episodes

json_dir='./json_logs'
if not os.path.exists(json_dir):
    os.makedirs(json_dir)
print('record json dir:', json_dir)

png_dir='./png_logs'
if not os.path.exists(png_dir):
    os.makedirs(png_dir)
print('record png dir:', png_dir)


#for each step of environment...

env.render(record_pngs_dir=png_dir, record_json_dir=json_dir)
#run this only at the end of an episode to gather json steps
pommerman.utility.join_json_state(json_dir, ['agent0name','agent1name','agent2name','agent3name'], finished_at=0, config=env_id, info=info)

# baseline example code

In [ ]:
import gym

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines import PPO2

# multiprocess environment
n_cpu = 1
env = DummyVecEnv([lambda: gym.make('CartPole-v1') for i in range(n_cpu)])

model = PPO2(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=25000)
model.save("ppo2_cartpole")

del model # remove to demonstrate saving and loading

model = PPO2.load("ppo2_cartpole")

# Enjoy trained agent
obs = env.reset()



In [ ]:
print(obs)
print(env.buf_obs[None].shape)
print(env.observation_space)

action, _states = model.predict(obs)
obs, rewards, dones, info = env.step(action)

print(obs)
print(rewards)
print(dones)
print(info)

# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = env.step(action)
#     env.render()